In [4]:
from binance.client import Client
from binance import ThreadedWebsocketManager
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
api_key = 'haWzm6Ujj0SdpaCeCBiyyAvTvS78odvplgl4W9t29CVgSq5LrAHpt940JYQCt5TD'
secret_key = 'SgoyFqlJ6SvCYSXPFsU1ehPzQHhsfFV3Pa5scLF6y0stOO0aYXpfig6Fy4N0ePWX'
client = Client(api_key=api_key, api_secret=secret_key, tld='com')
account = client.get_account()

In [5]:
now = datetime.utcnow()
now

datetime.datetime(2022, 9, 12, 19, 32, 54, 650234)

In [7]:
past = now - timedelta(days=2)
past

datetime.datetime(2022, 9, 10, 19, 32, 54, 650234)

In [8]:
def get_most_recent(symbol, interval, days):
    
    now = datetime.utcnow()
    past = str(now - timedelta(days = days))
    
    bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                        start_str = past, end_str = None, limit = 1000)
    df = pd.DataFrame(bars)
    df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
    df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                  "Clos Time", "Quote Asset Volume", "Number of Trades",
                  "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
    df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
    df.set_index("Date", inplace = True)
    for column in df.columns:
        df[column] = pd.to_numeric(df[column], errors = "coerce")
    df["Complete"] = [True for row in range(len(df)-1)] + [False]
    
    return df

In [9]:
df = get_most_recent('BTCUSDT','1m',2)
df

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2022-09-10 19:35:00,21498.95,21580.00,21498.37,21559.79,727.60929,True
2022-09-10 19:36:00,21559.78,21595.58,21558.61,21576.40,413.81849,True
2022-09-10 19:37:00,21578.12,21614.09,21566.32,21610.07,566.13390,True
2022-09-10 19:38:00,21610.07,21626.41,21561.84,21575.02,533.57596,True
2022-09-10 19:39:00,21573.66,21577.72,21553.36,21566.87,215.88280,True
...,...,...,...,...,...,...
2022-09-12 19:30:00,22395.07,22397.40,22383.01,22390.40,186.23691,True
2022-09-12 19:31:00,22389.96,22407.99,22387.21,22401.60,183.37571,True
2022-09-12 19:32:00,22401.64,22420.58,22400.54,22414.90,178.70016,True


In [10]:
get_most_recent('BTCUSDT','1m',2/24)

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2022-09-12 17:38:00,22247.52,22252.96,22230.00,22245.30,226.08897,True
2022-09-12 17:39:00,22246.19,22246.72,22225.00,22233.65,175.92233,True
2022-09-12 17:40:00,22233.65,22241.77,22230.48,22235.60,130.34033,True
2022-09-12 17:41:00,22234.44,22235.60,22215.44,22217.49,174.17943,True
2022-09-12 17:42:00,22217.49,22235.00,22212.46,22229.41,206.67129,True
...,...,...,...,...,...,...
2022-09-12 19:33:00,22413.80,22417.58,22391.23,22391.23,173.72375,True
2022-09-12 19:34:00,22391.23,22400.12,22380.00,22390.25,243.08408,True
2022-09-12 19:35:00,22390.25,22394.16,22381.15,22383.07,141.31470,True


In [11]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length):
        self.symbol = symbol
        self.bar_length = bar_length
        # self.data = pd.DataFrame(columns = ["Open", "High", "Low", "Close", "Volume", "Complete"])
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days) # NEW!
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days): # NEW!
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df # Create self.data
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print("Time: {} | Price: {}".format(event_time, close))
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]

In [16]:
trader = LongOnlyTrader('BTCUSDT', '1m')
trader

In [17]:
trader.start_trading(historical_days=2)

Time: 2022-09-12 19:42:41.024000 | Price: 22419.24
Time: 2022-09-12 19:42:43.025000 | Price: 22419.02
Time: 2022-09-12 19:42:45.030000 | Price: 22425.89
Time: 2022-09-12 19:42:47.050000 | Price: 22429.15
Time: 2022-09-12 19:42:49.086000 | Price: 22429.97
Time: 2022-09-12 19:42:51.099000 | Price: 22430.18
Time: 2022-09-12 19:42:53.157000 | Price: 22428.78
Time: 2022-09-12 19:42:55.162000 | Price: 22428.8
Time: 2022-09-12 19:42:57.178000 | Price: 22424.78
Time: 2022-09-12 19:42:59.205000 | Price: 22422.05
Time: 2022-09-12 19:43:00.003000 | Price: 22424.1


In [18]:
trader.twm.stop()

Time: 2022-09-12 19:43:28.516000 | Price: 22431.01


In [20]:
trader.data

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2022-09-10 19:43:00,21592.04,21600.00,21583.91,21595.98,252.80022,True
2022-09-10 19:44:00,21595.99,21598.15,21586.73,21587.99,115.32518,True
2022-09-10 19:45:00,21587.90,21601.60,21565.12,21568.81,229.81438,True
2022-09-10 19:46:00,21568.81,21599.20,21567.55,21596.02,216.81755,True
2022-09-10 19:47:00,21594.71,21604.06,21594.28,21600.15,138.06608,True
...,...,...,...,...,...,...
2022-09-12 19:39:00,22405.57,22409.68,22391.22,22405.13,191.73247,True
2022-09-12 19:40:00,22406.54,22408.90,22396.09,22405.90,205.11592,True
2022-09-12 19:41:00,22405.90,22415.00,22400.12,22408.92,182.94378,True
